In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

data = gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to /home/ayush/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

2024-08-21 16:06:29.087018: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 16:06:29.154394: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 16:06:29.223507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 16:06:29.292263: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 16:06:29.309450: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 16:06:29.430341: I tensorflow/core/platform/cpu_feature_gu

In [3]:
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [4]:
inputsequences = []
for line in text.split('\n'):
    tokenlist = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(tokenlist)):
        n_gram_sequence = tokenlist[:i+1]
        inputsequences.append(n_gram_sequence)

In [5]:
inputsequences

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [6]:
max_sequence_len = max([len(x) for x in inputsequences])
inputsequences = np.array(pad_sequences(inputsequences, maxlen=max_sequence_len, padding='pre'))

In [7]:
inputsequences = np.array(pad_sequences(inputsequences, maxlen=max_sequence_len, padding='pre'))
inputsequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [8]:
import tensorflow as tf
x,y = inputsequences[:,:-1],inputsequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM,Dense,Dropout,GRU

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/home/ayush/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(x_train, y_train, epochs=100,validation_data=(x_test,y_test), verbose=1)

Epoch 1/100


2024-08-21 16:06:34.913538: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 396714120 exceeds 10% of free system memory.


642/644 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0275 - loss: 7.1882

2024-08-21 16:06:55.800424: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 99192984 exceeds 10% of free system memory.


644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - accuracy: 0.0275 - loss: 7.1870 - val_accuracy: 0.0336 - val_loss: 6.6984
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.0366 - loss: 6.4709 - val_accuracy: 0.0424 - val_loss: 6.7909
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.0416 - loss: 6.3254 - val_accuracy: 0.0480 - val_loss: 6.8048
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step - accuracy: 0.0505 - loss: 6.1807 - val_accuracy: 0.0488 - val_loss: 6.8159
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - accuracy: 0.0531 - loss: 6.0559 - val_accuracy: 0.0565 - val_loss: 6.8412
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.0606 - loss: 5.9197 - val_accuracy: 0.0604 - val_loss: 6.9181
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.0682 - loss: 5.7642 - val_accuracy: 0.0670 - val_loss: 6.9050
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.0770 - loss: 5.6233 - val_

KeyboardInterrupt: 

In [ ]:
def predict_next_word(seed_text, model, tokenizer, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    if len(token_list) >= max_sequence_len:
      token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted_probs,axis=1)
    for word ,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None


In [ ]:
input_text = 'To be or not to be'
print(input_text)
max_sequence_len = model.input_shape[1]+1
next_word=predict_next_word(input_text,model,tokenizer,max_sequence_len)
print(next_word)

In [ ]:
model.save("next_word_lstm.h5")
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)